In [20]:
from importlib import resources
import sys
import inspect
from pathlib import Path
import traceback
import tempfile, os, shutil, time
import signal

from kinoml.core.ligands import Ligand
from kinoml.core.proteins import Protein, KLIFSKinase
from kinoml.core.systems import ProteinLigandComplex
from kinoml.features.core import Pipeline
from kinoml.features.complexes import (
    MostSimilarPDBLigandFeaturizer,
    KLIFSConformationTemplatesFeaturizer,
    OEDockingFeaturizer,
)

from rdkit import Chem
import pandas as pd
import requests as req
from multiprocessing import Pool, cpu_count
import tqdm
import numpy as np
import dask
import dask.dataframe as dd
import MDAnalysis as mda

# Reference results

In [21]:
DATA_DIR = Path("../docking")
similar = pd.read_csv(
    DATA_DIR / "similar.csv",
    names="ident uniprot smiles pdb chain ligand_pdb".split(),
    index_col="ident",
)
pdbs = pd.read_csv(DATA_DIR / "pdbs.csv", names="ident structure_id".split(), index_col="ident")
data = similar.join(pdbs, on="ident", how="inner")
activities = pd.read_csv(DATA_DIR / "activities-chembl31.csv", index_col="activities.activity_id")
activities.drop(columns=["Unnamed: 0"], inplace=True)
activities.index.name = "ident"
data = data.join(activities, on="ident", how="inner")

In [22]:
NPARTITION = 10
TIMEOUT = 60  # in sec

# directories
HERE = Path(".").absolute()
CACHE_DIR = HERE / "docking_pipeline"
MOST_SIMILAR = CACHE_DIR / "most_similar.csv"
KLIFS_DIR = CACHE_DIR / "KLIFS"
KLIFS_MAP = CACHE_DIR / "similar_klifs_structures.csv"
TEMP_DIR = CACHE_DIR / "temporary"
DOCKING_DIR = CACHE_DIR / "docking"

In [23]:
kinodata = pd.read_csv("data/activities-chembl31.csv", index_col=0)

In [28]:
kinodata["activities.activity_id"].unique().shape

(202592,)

In [5]:
kinodata = kinodata.head(5)

# Find most similar PDBs

In [29]:
d = pd.read_csv("most_similar.csv")
d

,activities.activity_id,similar.ligand_pdb,similar.complex_pdb,similar.chain
0,16291323,VFS,7JOV,A
1,16306943,5YS,7P6N,A
2,16264754,J0P,6ED6,A
3,16340050,J0P,6ED6,A
4,16340956,J0P,6ED6,A
...,...,...,...,...
221259,16276373,1SW,4KSQ,A
221260,16347116,QF8,6ZCY,A
221261,16339375,734,4E26,A
221262,19334839,ESW,6G9M,A


In [30]:
d["activities.activity_id"].unique().shape

(64,)

In [25]:
protein = KLIFSKinase(uniprot_id="O75116", toolkit="OpenEye")
ligand = Ligand(smiles="CCCC(=O)Nc1cccc(-c2nc(Nc3ccc4[nH]ncc4c3)c3cc(OCCN4CCC(O)C4)ccc3n2)c1")
system = ProteinLigandComplex(components=[protein, ligand])

klifs_featurizer = KLIFSConformationTemplatesFeaturizer(
    similarity_metric="fingerprint",
    cache_dir=CACHE_DIR,
)

docking_featurizer = OEDockingFeaturizer(output_dir=DOCKING_DIR, method="Posit")

featurizer = Pipeline([klifs_featurizer, docking_featurizer])

systems = featurizer.featurize([system])

# system.featurizations['last']

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 30, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 29, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 31, aromatic: False, chiral: False
Atom atomic num: 6, name: , idx: 33, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 32, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 34, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 67, aromatic: False, chiral: False

Cou

In [13]:
system.featurizations["last"].to_csv("test.csv")

In [17]:
def get_system(args):
    ident, uniprot_id, ligand_smiles = args
    # try:
    protein = KLIFSKinase(uniprot_id=uniprot_id, toolkit="MDAnalysis")
    ligand = Ligand(smiles=ligand_smiles)
    system = ProteinLigandComplex(components=[protein, ligand])

    return system


def get_most_similar(args):
    featurizer = MostSimilarPDBLigandFeaturizer(
        similarity_metric="fingerprint",
        cache_dir=CACHE_DIR,
    )

    system = featurizer.featurize([system])[0]

    ligand_id = system.protein.expo_id
    pdb_id = system.protein.pdb_id
    chain = system.protein.chain_id

    return ligand_id, pdb_id, chain

    # with open(MOST_SIMILAR, "a") as f:
    #     f.write(",".join(map(str, [ident, ligand_id, pdb_id, chain])) + "\n")
    # return 0
    # except:
    #     return 1

In [41]:
with open(MOST_SIMILAR, "w") as f:
    f.write("activities.activity_id,similar.ligand_pdb,similar.complex_pdb,similar.chain\n")

In [18]:
systems = list()
for _, row in kinodata.iterrows():
    uniprot_id = row["UniprotID"]
    ligand_smiles = row["compound_structures.canonical_smiles"]
    ident = row["activities.activity_id"]
    systems.append(get_system((ident, uniprot_id, ligand_smiles)))

In [19]:
CHUNKSIZE = 100
# with open(MOST_SIMILAR, "w") as f:
#     f.write("activities.activity_id,similar.ligand_pdb,similar.complex_pdb,similar.chain\n")
featurized_systems = list()
for i in tqdm.tqdm(range(0, len(systems), CHUNKSIZE)):
    featurizer = KLIFSConformationTemplatesFeaturizer(
        similarity_metric="fingerprint",
    )
    featurized_systems += featurizer.featurize(systems[i : i + CHUNKSIZE])
    for i, system in enumerate(featurized_systems):
        system.featurizations["last"].to_csv(f"{i}.csv")

  0%|                                                                                                                                                                    | 0/1 [00:00<?, ?it/s]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 30, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 29, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 31, aromatic: False, chiral: False
Atom atomic num: 6, name: , idx: 33, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 32, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 34, aromatic: False, chiral: False
bond order: 1

# Download KLIFS structures

In [8]:
ms = pd.read_csv(MOST_SIMILAR)

In [9]:
kinodata = ms.merge(kinodata, how="inner", on="activities.activity_id")

In [10]:
# kinodata = dd.from_pandas(kinodata, npartitions=NPARTITION)

In [43]:
def klifs_pdb(
    ident,
    pdb_id: str,
    chain: str,
    ligand_id: str,
    output_path=Path(".").absolute(),
    lazy: bool = False,
):
    """
    Get the complex PDB from KLIFS.

    Parameters
    ----------
    pdb_id: str
        PDB ID.
    chain: str
        the chain.
    ligand_id: str
        ligand PDB ID
    path: Path, optional
        folder to store the structure in.

    Returns
    -------
    file_path: Path
        path of the PDB file.
    """
    resp = req.get("https://klifs.net/api_v2/structures_pdb_list", {"pdb-codes": pdb_id})
    klifs_info = None
    resp.raise_for_status()
    for info in resp.json():
        if (
            str(info["chain"]).upper() == str(chain).upper()
            and str(info["ligand"]).upper() == str(ligand_id).upper()
        ):
            if klifs_info is None:
                klifs_info = info
            elif klifs_info["quality_score"] < info["quality_score"]:
                klifs_info = info
    if klifs_info is None:
        raise ValueError(f"not found pdb:{pdb_id} chain:{chain} lig_pdb:{ligand_id}")
    structure_ID = klifs_info["structure_ID"]
    output_path.mkdir(exist_ok=True)
    filename = output_path / f"{structure_ID}.pdb"
    if not lazy or not filename.exists():
        resp = req.get(
            "https://klifs.net/api_v2/structure_get_pdb_complex", {"structure_ID": structure_ID}
        )

    with open(filename, "w") as f:
        f.write(resp.text)

    with open(KLIFS_MAP, "a") as f:
        f.write(f"{ident},{structure_ID}\n")

    return 0

In [ ]:
e = val_errs[0][1]

In [39]:
# http_errs = list()
val_errs = list()
success = 0
for x in lost_ids:
    print((success, len(http_errs), len(val_errs)), end="\r", flush=True)
    entry = similar.loc[x]
    try:
        klifs_pdb(x, entry.pdb, entry.chain, entry.ligand_pdb)
        success += 1
    except req.HTTPError as e:
        if e.args[0].startswith("400 Client Error: Bad Request for url"):
            http_errs.append(x)
        else:
            raise e
    except ValueError as e:
        if e.args[0].startswith("not found pdb:"):
            val_errs.append((x, e))
        else:
            raise e

In [62]:
with open("missing_pdbs", "w") as f:
    for x in np.unique(similar.loc[http_errs].pdb.values):
        f.write(f"{x}\n")

In [127]:
u = sys.featurizations["last"]
with MDAnalysis.Writer("universe.pdb", multiframe=True) as pdb:
    # for ts in u.trajectory:
    pdb.write(u.select_atoms("resname LIG"))

/home/michael/.miniconda3/envs/kinoml/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:753: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/michael/.miniconda3/envs/kinoml/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


In [73]:
with open("missing_ligands.csv", "w") as f:
    f.write("PDB,chain,ligand_PDB\n")
    for i, row in (
        similar.loc[[x[0] for x in val_errs]]
        .groupby(["pdb", "chain", "ligand_pdb"])
        .agg(len)
        .iterrows()
    ):
        f.write(",".join(i) + "\n")

In [74]:
d = pd.read_csv("missing_ligands.csv")

In [90]:
with open("invalid_ligand_kinases.csv", "w") as f:
    for pdb_id in tqdm.tqdm(d.PDB.unique()):
        resp = req.get("https://klifs.net/api_v2/structures_pdb_list", {"pdb-codes": pdb_id})
        klifs_info = None
        resp.raise_for_status()

        for info in resp.json():
            if info["ligand"] == 0:
                f.write(f"{info['structure_ID']}\n")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 217/217 [00:40<00:00,  5.30it/s]


In [12]:
with open(KLIFS_MAP, "w") as f:
    f.write("activities.activity_id,similar.klifs_structure_id\n")

In [13]:
jobs = list()
for _, row in kinodata.iterrows():
    pdb_id = row["similar.complex_pdb"]
    ligand_pdb = row["similar.ligand_pdb"]
    chain = row["similar.chain"]
    ident = row["activities.activity_id"]
    jobs.append(klifs_pdb(ident, pdb_id, chain, ligand_pdb, output_path=KLIFS_DIR, lazy=False))

# OpenEye Template docking

In [14]:
klifs_map = pd.read_csv(KLIFS_MAP)
kinodata = klifs_map.merge(kinodata, how="inner", on="activities.activity_id")

In [15]:
class temporary_copy(object):
    def __init__(self, original_path):
        self.original_path = original_path

    def __enter__(self):
        temp_dir = tempfile.gettempdir()
        base_path = os.path.basename(self.original_path)
        self.path = os.path.join(temp_dir, base_path)
        shutil.copy2(self.original_path, self.path)
        return self.path

    def __exit__(self, exc_type, exc_val, exc_tb):
        os.remove(self.path)

In [16]:
def handler(signum, frame):
    print("Stop it already!")
    raise Exception("timeout")

In [17]:
# @dask.delayed
def posit(ident, pdb_filepath, ligand_smiles, output_dir):
    print(pdb_filepath)
    protein = Protein.from_file(pdb_filepath)
    ligand = Ligand(smiles=ligand_smiles)
    system = ProteinLigandComplex(components=[protein, ligand])
    featurizer = OEDockingFeaturizer(
        output_dir=output_dir, method="Posit", use_multiprocessing=False
    )

    signal.signal(signal.SIGALRM, handler)
    signal.alarm(TIMEOUT)
    try:
        system = featurizer.featurize([system])[0]
    except Exception as exc:
        print(exc)
        return 1

    print("write result")
    universe = system.featurizations["last"]
    docking_score = universe._topology.docking_score
    posit_probability = universe._topology.posit_probability

    print("write", output_dir / f"{ident}_complex.pdb")
    with mda.coordinates.PDB.PDBWriter(output_dir / f"{activity_id}_complex.pdb") as w:
        w.write(universe)
    with mda.coordinates.PDB.PDBWriter(output_dir / f"{activity_id}_ligand.pdb") as w:
        w.write(universe.select_atoms("resname LIG"))

    with open(output_dir / "docking.csv", "a") as f:
        f.write(
            ",".join(list(map(str, [ident, docking_score, posit_probability, duration]))) + "\n"
        )
    return 0

In [20]:
jobs = list()
for _, row in kinodata.iterrows():
    ident = row["activities.activity_id"]
    pdb_filepath = KLIFS_DIR / (str(row["similar.klifs_structure_id"]) + ".pdb")
    ligand_smiles = row["compound_structures.canonical_smiles"]
    jobs.append(posit(ident, pdb_filepath, ligand_smiles, DOCKING_DIR))

/home/michael/Code/kinodata-docking/docking_pipeline/KLIFS/12757.pdb


OEDockingFeaturizer:   0%|          | 0/1 [00:00<?, ?it/s]

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 30, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 29, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 31, aromatic: False, chiral: False
Atom atomic num: 6, name: , idx: 33, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 32, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 34, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 35, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 67, aromatic: False, chiral: False



Stop it already!
timeout
/home/michael/Code/kinodata-docking/docking_pipeline/KLIFS/14314.pdb


OEDockingFeaturizer:   0%|          | 0/1 [00:00<?, ?it/s]

...

Stop it already!
timeout
/home/michael/Code/kinodata-docking/docking_pipeline/KLIFS/10689.pdb


OEDockingFeaturizer:   0%|          | 0/1 [00:00<?, ?it/s]

Stop it already!
timeout
/home/michael/Code/kinodata-docking/docking_pipeline/KLIFS/10689.pdb


OEDockingFeaturizer:   0%|          | 0/1 [00:00<?, ?it/s]

Stop it already!
timeout
